# Exploration of LLM for election

In [33]:
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import faiss
import numpy as np
import ollama

## Open documents

In [2]:
def extract_all_text(pdf_path):
    # Open the PDF file with a 'with' statement
    with pdfplumber.open(pdf_path) as pdf:
        full_text = ""
        # Loop through each page in the PDF
        for page in pdf.pages:
            # Extract the text from the current page
            text = page.extract_text()
            if text:
                full_text += text + "\n"
    return full_text

In [3]:
def extract_text_with_pdfplumber(pdf_path, txt_path):
    with pdfplumber.open(pdf_path) as pdf:
        with open(txt_path, "w", encoding="utf-8") as f:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    f.write(text + '\n')

In [4]:
text_vvd = extract_all_text(r"C:\Users\Sten\Documents\projects\electionLLM\programma\Verkiezingsprogramma-TK-VVD2025.pdf")

## Create chunks

In [5]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,       # maximaal 500 tokens per chunk
    chunk_overlap=100,    # overlap tussen chunks om context te behouden
    separators=["\n\n", "\n", ".", " "]  # logische splitsingen
)

In [8]:
chunks = splitter.split_text(text_vvd)

print(f"Totaal aantal chunks: {len(chunks)}")
print("Voorbeeld chunk:")
print(chunks[1])

Totaal aantal chunks: 729
Voorbeeld chunk:
ter wereld gemaakt. Om die vrijheid en veiligheid te behouden, is actieve bescherming van
de liberale democratie en de democratische rechtsstaat noodzakelijk. De liberale waarden
die daaraan ten grondslag liggen, zijn voor ons dan ook niet onderhandelbaar. Vrijheid,
verantwoordelijkheid, verdraagzaamheid, gelijkwaardigheid en sociale rechtvaardigheid vormen
de rotsvaste bouwstenen van onze maatschappij. Deze liberale waarden zijn niet alleen in het


## Create embedding

In [19]:
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
#embedding_model = SentenceTransformer("intfloat/e5-base")

In [ ]:
embeddings = []

for doc in tqdm(chunks, desc="Embedding chunks"):
    emb = embedding_model.encode(doc)
    embeddings.append(emb)

In [20]:
chunk_embeddings = embedding_model.encode(chunks, convert_to_numpy=True)

print(f"Vorm van embedding matrix: {chunk_embeddings.shape}")

Vorm van embedding matrix: (729, 384)


In [25]:
index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
index.add(chunk_embeddings)
print(f"Aantal vectoren in index: {index.ntotal}")

Aantal vectoren in index: 729


## Create LLM output

In [53]:
question = "wat vind de vvd van immigratie"
question_embedding = embedding_model.encode(question, convert_to_numpy=True)
question_embedding = np.array(question_embedding).reshape(1, -1)

D, I = index.search(question_embedding, k=2)

retrieved_context = "\n\n".join([chunks[idx] for idx in I[0]])

prompt = f"""
Beantwoord de vraag op basis van de onderstaande fragmenten uit partijprogramma's.
Geef altijd aan welke partij iets zegt en verwijs letterlijk naar het fragment met "".

Vraag: {question}

Fragmenten:
{retrieved_context}

Antwoord:
"""

stream = ollama.chat(model='mistral', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for piece in stream:
    print(piece['message']['content'], end='', flush=True)


 De VVD vindt dat er rechtsbijstand aan asielzoekers wordt gegeven en dat de migratieketen versterkt moet worden. Ze stelt dat het IND, COA en de Dienst Terugkeer en Vertrek extra capaciteit krijgen om zaken snel af te handelen, zodat duidelijkheid komt over wie hier wel en wie hier niet mag blijven. Deze maatregelen zijn bedoeld om een humane opvang voor echte vluchtelingen mogelijk te maken, maar niet ten koste van eigen inwoners.

In [54]:
retrieved_context

'aan onze eigen veiligheid, vrijheid en economische kracht. Dit geldt dus ook voor de\nmiddelen die we inzetten voor ontwikkelingssamenwerking. Migratiebeheersing is ook\ngeopolitiek, en vormt daarom een cruciaal onderdeel van dit beleid. Landen die weigeren om\nuitgeprocedeerde asielzoekers terug te nemen, kunnen niet langer op onze hulp rekenen.\nOnze steun is voorwaardelijk en dient altijd het Nederlands belang.\n▶ Ontwikkelingssamenwerking meer in EU-verband: Als VVD plaatsen we al jaren\n\nrechtsbijstand voor asielzoekers.\n▶ De migratieketen wordt versterkt: De IND krijgt voldoende capaciteit om zaken snel af\nte handelen, zodat er snel duidelijkheid komt over wie hier wel en wie hier niet mag blijven.\nHetzelfde geldt voor het COA. De Dienst Terugkeer en Vertrek krijgt hierbij extra aandacht,\nwaardoor het terugkeerbeleid kan worden geïntensiveerd.\nHumane opvang voor echte vluchtelingen, maar\nniet ten koste van eigen inwoners'